In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

import psycopg2
import pandas.io.sql as sqlio
import psycopg2.extras


In [3]:
# Establish a connection to the PLUTO database
try:
    conn = psycopg2.connect("dbname='pluto' user='oliver' host='localhost' password='m'")
except:
    print("I am unable to connect to the database")

# Create a cursor object for executing queries
conn.autocommit = True
cur = conn.cursor()

In [4]:
    cov_forest = [
    'xcoord'
    ,'ycoord'
    ,'age'
    ,'build_alter_gap'
    ,'alterage'      
    ,'elevator'
    ,'commercial'
    ,'garage'
    ,'storage'
    ,'basement'
    ,'waterfront'
    ,'frontage'
    ,'block_assmeblage'
    ,'cooperative'
    ,'conv_loft_wh'
    ,'tenament'
    ,'garden'
    ,'semi_fireproof'
    ,'fireproof'
    ,'luxury'
    ,'artist_res'
    ,'units_per_building'
    ,'floors'
    ,'zip_avg_num_buildings'
    ,'pcvpu_l1'
    ,'pcvpu_l2'
    ,'pcvpu_l3'
    ,'avg_block_pcvpu_l1'
    ,'avg_block_pcvpu_l2'
    ,'avg_block_pcvpu_l3'
    ,'avg_neighbor1_pcvpu_l1'
    ,'avg_neighbor1_pcvpu_l2'
    ,'avg_neighbor1_pcvpu_l3'
    ,'avg_neighbor2_pcvpu_l1'
    ,'avg_neighbor2_pcvpu_l2'
    ,'avg_neighbor2_pcvpu_l3'
    ,'avg_neighbor3_pcvpu_l1'
    ,'avg_neighbor3_pcvpu_l2'
    ,'avg_neighbor3_pcvpu_l3'
    ,'avg_neighbor4_pcvpu_l1'
    ,'avg_neighbor4_pcvpu_l2'
    ,'avg_neighbor4_pcvpu_l3'
    ,'avg_zip_pcvpu_l1'
    ,'avg_zip_pcvpu_l2'
    ,'avg_zip_pcvpu_l3'
    ]

In [5]:
# Commands to select data from each year
sql_2008 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2008
         """
sql_2009 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2009
         """
sql_2010 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2010
         """
sql_2010 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2010
         """
sql_2011 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2011
         """
sql_2012 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2012
         """
sql_2013 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2013
         """
sql_2014 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2014
         """
sql_2015 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2015
         """
sql_2016 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2016
         """
sql_2017 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2017
         """
sql_2018 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2018
         """
sql_2019 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2019
         """
sql_2020 = """SELECT * 
         FROM pluto_sym
         WHERE year = 2020
         """

In [6]:
# Function for creating dataframes from a SQL query
def get_data(sql_train, sql_test, cov_forest):
    """
    Function to return data from a train and test SQL query
    """
    global df_train, df_test, X_train, X_test, y_train, y_test, covariates

    # List of columns to be used as covariates in the model

    df_train = sqlio.read_sql_query(sql_train, conn).dropna()
    df_test = sqlio.read_sql_query(sql_test, conn).dropna()

    covariates = df_train.columns.intersection(cov_forest)

    X_train = df_train[covariates]
    y_train = df_train['pcvpu_target']

    X_test = df_test[covariates]
    y_test = df_test['pcvpu_target']
    return df_train, df_test, X_train, X_test, y_train, y_test

In [10]:
reg = LinearRegression()

In [14]:
# Set up empty lists to store the results of the model for each year

MAEs = [] ## Mean Absolute Error, list of numbers (out of sample)
MSEs = [] ## Mean Squared Error, list of numbers (out of sample)
R2s = []  ## R2 score, list of numbers (out of sample)

feature_importances = [] ## Feature importances, list of lists

MAEs_in_sample = [] ## Mean Absolute Error, list of numbers (in sample)
MSEs_in_sample = [] ## Mean Squared Error, list of numbers (in sample)
R2s_in_sample = []  ## R2 score, list of numbers (in sample)


def update_reports():
    """
    Function to update the reports for each year
    """
    print('Updating Out of Sample Reports')

    MSEs.append(mean_squared_error(y_test, y_pred))
    MAEs.append(mean_absolute_error(y_test, y_pred))
    R2s.append(r2_score(y_test, y_pred))
    print('MSE: {}'.format(MSEs[-1]))
    print('MAE: {}'.format(MAEs[-1]))
    print('R2: {}'.format(R2s[-1]))

    print('Updating In Sample Reports')

    MSEs_in_sample.append(mean_squared_error(y_train, y_pred_in_sample))
    MAEs_in_sample.append(mean_absolute_error(y_train, y_pred_in_sample))
    R2s_in_sample.append(r2_score(y_train, y_pred_in_sample))
    print('MSE: {}'.format(MSEs_in_sample[-1]))
    print('MAE: {}'.format(MAEs_in_sample[-1]))
    print('R2: {}'.format(R2s_in_sample[-1]))



In [8]:
# get data from 2008 and 2009
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2008, sql_2009, cov_forest)


In [11]:
# fit the model for 2008 and 2009
reg.fit(X_train, y_train)

LinearRegression()

In [12]:
# Make in and out of sample predictions
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)

In [15]:
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 322.5502054027073
MAE: 6.766008101200189
R2: -0.003665986023268397
Updating In Sample Reports
MSE: 413.0932977984302
MAE: 7.484008769872224
R2: 0.13985880083016067


In [16]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2009, sql_2010, cov_forest)

In [17]:
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)

LinearRegression()

In [18]:
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)

In [19]:
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 212.93921704843558
MAE: 6.4108073640719585
R2: -0.05978951698428414
Updating In Sample Reports
MSE: 300.2939845854991
MAE: 6.194425713383385
R2: 0.06558776560205093


In [20]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2010, sql_2011, cov_forest)

In [21]:
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)

LinearRegression()

In [22]:
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)

In [23]:
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 34.07054158149536
MAE: 2.8748552348057057
R2: -0.17495045435023915
Updating In Sample Reports
MSE: 194.29466196103434
MAE: 5.751134322943571
R2: 0.03300364861643912


In [24]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2011, sql_2012, cov_forest)

In [25]:
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)

LinearRegression()

In [26]:
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)

In [27]:
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 144.73653142962962
MAE: 5.940657283158737
R2: -0.07252823431945288
Updating In Sample Reports
MSE: 28.437546268814266
MAE: 2.3079466134939595
R2: 0.019307989888349142


In [28]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2012, sql_2013, cov_forest)

In [29]:
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)

LinearRegression()

In [30]:
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)

In [31]:
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 131.65027681111658
MAE: 4.966707639178221
R2: 0.02598813352736773
Updating In Sample Reports
MSE: 124.39158028455287
MAE: 4.7739098902222485
R2: 0.07823214603105844


In [32]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2013, sql_2014, cov_forest)
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)
# make predictions for the next year
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 105.44099038519488
MAE: 4.526180668620746
R2: -0.026123434931326628
Updating In Sample Reports
MSE: 128.34495216113584
MAE: 4.863862922748104
R2: 0.05044250999818045


In [33]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2014, sql_2015, cov_forest)
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)
# make predictions for the next year
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 126.67433475781401
MAE: 5.106091559674239
R2: 0.011323357648602816
Updating In Sample Reports
MSE: 98.21598688072756
MAE: 4.211165190550344
R2: 0.044188361138789434


In [34]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2015, sql_2016, cov_forest)
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)
# make predictions for the next year
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 92.99246113022777
MAE: 4.240718315983647
R2: 0.006261701416020005
Updating In Sample Reports
MSE: 124.39520531416538
MAE: 5.093968492508225
R2: 0.029111665360175176


In [35]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2016, sql_2017, cov_forest)
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)
# make predictions for the next year
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 109.61138353477074
MAE: 3.966351512409227
R2: 0.017161873048872622
Updating In Sample Reports
MSE: 91.36458439637161
MAE: 3.996974708868859
R2: 0.02365755733966579


In [36]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2017, sql_2018, cov_forest)
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)
# make predictions for the next year
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 8882.538268734292
MAE: 75.73188354371494
R2: -0.01214639817722074
Updating In Sample Reports
MSE: 109.13548611412628
MAE: 3.906090949441629
R2: 0.021429040513086806


In [37]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2018, sql_2019, cov_forest)
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)
# make predictions for the next year
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 3705.898652107344
MAE: 44.40687699933639
R2: -18.53697885694937
Updating In Sample Reports
MSE: 5666.172944622217
MAE: 55.89535897028559
R2: 0.3543516094340682


In [38]:
# get data for the next year
df_train, df_test, X_train, X_test, y_train, y_test = get_data(sql_2019, sql_2020, cov_forest)
# expand the number of trees and then fit to the next year

reg.fit(X_train,y_train)
# make predictions for the next year
y_pred = reg.predict(X_test)
y_pred_in_sample = reg.predict(X_train)
# update all the report series
update_reports()

Updating Out of Sample Reports
MSE: 328.69834250404733
MAE: 14.59105348630783
R2: -1.0745421733875897
Updating In Sample Reports
MSE: 184.2303933081035
MAE: 8.90645704603528
R2: 0.028763159289001305


In [1]:
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
years_in_sample = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
plt.plot(years, R2s, label='Out of Sample')
plt.plot(years_in_sample, R2s_in_sample, label='In Sample')
plt.legend()
plt.xlabel('Year')
plt.ylabel('R2 Score')
plt.title('R2 Scores by Year')


NameError: name 'plt' is not defined

In [2]:
plt.plot(years, MSEs, label='Out of Sample')
plt.plot(years_in_sample, MSEs_in_sample, label='In Sample')
plt.legend()
plt.xlabel('Year')
plt.ylabel('MSE')
plt.title('Mean Squared Error by Year')

NameError: name 'plt' is not defined